In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import xmltodict
import time

In [6]:
authKey = "G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D"

## 전체 아파트 목록 조회 함수
def get_apt_list(authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/AptListService2/getTotalAptList?serviceKey={authKey}&pageNo=1&numOfRows=1000000"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    raw = pd.DataFrame(raw['response']['body']['items']['item'])
    return raw

## 아파트 기본 정보 조회 함수
def get_danji_base_info(danji_cd, authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo?serviceKey={authKey}&kaptCode={danji_cd}"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    raw = pd.DataFrame(raw['response']['body']['item'], index=[0])
    return raw

## 아파트 상세 정보 조회 함수
def get_danji_detail_info(danji_cd, authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusDtlInfo?serviceKey={authKey}&kaptCode={danji_cd}"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    raw = pd.DataFrame(raw['response']['body']['item'], index=[0])
    return raw

In [19]:
danji_list = get_apt_list()
danji_list.columns = ['시도명', '시군구명', '읍면동명', '법정동코드', '단지코드', '단지명', '리명']

In [21]:
danji_cd_ls = danji_list[danji_list['법정동코드'] == "1121510400"]['단지코드'].to_list()

In [22]:
## 1차 획득
_ls = []
for danji_cd in danji_cd_ls:
    try:
        _ls.append(get_danji_base_info(danji_cd))
    except Exception as e:
        print(f"{danji_cd} : {e}")
base_info = pd.concat(_ls)

A14381510 : 'response'
A14381506 : 'response'
A14381508 : 'response'
A10024751 : 'response'


In [24]:
## 오류 부분 획득
err_ls = []
for danji_cd in [code for code in danji_cd_ls if code not in base_info['kaptCode'].to_list()]:
    try:
        err_ls.append(get_danji_base_info(danji_cd))
    except Exception as e:
        print(f"{danji_cd} : {e}")
base_info_err = pd.concat(err_ls)

In [25]:
## 저장
path = 'C:/Users/GCC/git/dataset/DL'
base_info_res = pd.concat([base_info, base_info_err])
base_info_res.to_csv(f"{path}/단지기본정보.csv", index=False)